<a href="https://colab.research.google.com/github/bwsi-hadr/2019-student-final-exercise/blob/master/Planning_Prep_Phase_Facility_Flooding_and_Set_Cover_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
# need to specify location of some certificates for rasterio
!export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt
!sudo mkdir -p /etc/pki/tls/certs
!sudo cp /etc/ssl/certs/ca-certificates.crt /etc/pki/tls/certs/ca-bundle.crt
try:
  import rasterio
  import rasterio.plot
  import rasterio.merge 
  import rasterio.mask
except:
  !pip install rasterio
  import rasterio
  import rasterio.plot
  import rasterio.merge
  import rasterio.mask
  
try:
  import rasterstats as rs
except:
  !pip install rasterstats  
  import rasterstats as rs
  
try:
  import pyproj
except:
  !pip install pyproj
  import pyproj
  
import networkx as nx
try:
  import osmnx as ox
except:
  # osmnx depends on the system package libspatialindex
  !apt install libspatialindex-dev
  !pip install osmnx
  import osmnx as ox

try: 
  import geopandas as gpd
except: 
  !pip install geopandas 
  import geopandas as gpd
  
try:
  import contextily as ctx 
except:
  # install dependencies for contextily
  !apt install libproj-dev proj-data proj-bin
  !apt install libgeos-dev
  !pip install cython
  !pip install cartopy
  # install contextily
  !pip install contextily==1.0rc1 --no-use-pep517 --no-cache-dir
  import contextily as ctx
  
import fiona
from shapely.geometry import Point, LineString, Polygon
  
import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/BWSI-Remote-Sensing/'Final_exercise'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/BWSI-Remote-Sensing/Final_exercise


In [181]:
!ls game_grid_export

facilities	      flooding.shx	       track.cpg
facilities_stats.csv  game_grid_all_stats.cpg  track.dbf
flooding.cpg	      game_grid_all_stats.dbf  track.prj
flooding.dbf	      game_grid_all_stats.prj  track.qpj
flooding.prj	      game_grid_all_stats.shp  track.shp
flooding.shp	      game_grid_all_stats.shx  track.shx


In [0]:
grid = gpd.read_file('game_grid_export/game_grid_all_stats.shp').to_crs(epsg = 3857)
grid['centroid'] = grid['geometry'].centroid

In [0]:
to_m = lambda km: km * 1000

#Hospitals

In [184]:
hospital_grid = gpd.read_file('game_grid_export/facilities/hospitals.shp').to_crs(grid.crs)
hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] >= 200, 'radius'] = 25
hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] < 200, 'radius'] = 15
hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] >= 200, 'type'] = "large hospital"
hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] < 200, 'type'] = "small hospital"
hospital_grid['health'] = 2

big_hospitals=hospital_grid[hospital_grid['BEDS']>=200].to_crs(grid.crs)
small_hospitals=hospital_grid[hospital_grid['BEDS']<200].to_crs(grid.crs)
hospitals_buffered = hospital_grid.copy().to_crs(grid.crs)
hospitals_buffered.loc[big_hospitals.index, 'geometry'] = hospitals_buffered.loc[big_hospitals.index,'geometry'].buffer(to_m(25))
hospitals_buffered.loc[small_hospitals.index, 'geometry'] = hospitals_buffered.loc[small_hospitals.index,'geometry'].buffer(to_m(15))
hospitals_buffered['centroid'] = hospitals_buffered['geometry'].centroid

hosp_info = gpd.sjoin(grid, hospitals_buffered, how = 'inner', op = 'intersects')
hosp_infopop = hosp_info.groupby('NAME').sum()

alpha_hosp = hospitals_buffered.sort_values('NAME').set_index('NAME')
alpha_hosp['population'] = hosp_infopop['population'].values
hosp_idx = alpha_hosp.reset_index()

alpha_hosp['population']

NAME
ARBOUR FULLER HOSPITAL                                      216673
ARBOUR HOSPITAL, THE                                        778984
ARBOUR HUMAN RESOURCE INSTITUTE                             869992
BAY RIDGE HOSPITAL BEVERLY                                  352378
BETH ISRAEL DEACONESS HOSPITAL - NEEDHAM                    330194
BETH ISRAEL DEACONESS HOSPITAL MILTON                       501945
BETH ISRAEL DEACONESS HOSPITAL PLYMOUTH                      38508
BETH ISRAEL DEACONESS MEDICAL CENTER - EAST CAMPUS         1208502
BETH ISRAEL DEACONESS MEDICAL CENTER - WEST CAMPUS         1200722
BOSTON MEDICAL CENTER - EAST NEWTON CAMPUS                 1216527
BOSTON MEDICAL CENTER - MENINO CAMPUS                      1217093
BOURNEWOOD HOSPITAL                                         712321
BRIGHAM AND WOMEN'S HOSPITAL                               1204412
CAMBRIDGE HEALTH ALLIANCE - CAMBRIDGE CAMPUS               1222459
CAMBRIDGE HEALTH ALLIANCE - SOMERVILLE CAMPUS            

#EMS

In [185]:
EMS_grid = gpd.read_file('game_grid_export/facilities/EMS.shp').to_crs(grid.crs)
EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] >= 5, 'radius'] = 15
EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] < 5, 'radius'] = 10
EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] >= 5, 'type'] = "large EMS"
EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] < 5, 'type'] = "small EMS"
EMS_grid['health'] = 1

big_EMS=EMS_grid[EMS_grid['TOTAL_VEHI']>=5].to_crs(grid.crs)
small_EMS=EMS_grid[EMS_grid['TOTAL_VEHI']<5].to_crs(grid.crs)
EMS_buffered = EMS_grid.copy().to_crs(grid.crs)
EMS_buffered.loc[big_EMS.index, 'geometry'] = EMS_buffered.loc[big_EMS.index,'geometry'].buffer(to_m(15))
EMS_buffered.loc[small_EMS.index, 'geometry'] = EMS_buffered.loc[small_EMS.index,'geometry'].buffer(to_m(10))
EMS_buffered['centroid'] = EMS_buffered['geometry'].centroid

EMS_info = gpd.sjoin(grid, EMS_buffered, how = 'inner', op = 'intersects')
EMS_infopop = EMS_info.groupby('NAME').sum()

alpha_EMS = EMS_buffered.sort_values('NAME').set_index('NAME')
alpha_EMS = alpha_EMS.loc[~alpha_EMS.index.duplicated(keep='first')]
alpha_EMS.loc[EMS_infopop.index,'population'] = EMS_infopop['population'].values
alpha_EMS = alpha_EMS.dropna(subset=['population'])
EMS_idx = alpha_EMS.reset_index()

alpha_EMS['population']

NAME
ABINGTON FIRE DEPARTMENT STATION 1                               100719.0
ABINGTON FIRE DEPARTMENT STATION 2                               183927.0
ACTION AMBULANCE SERVICE - HEADQUARTERS                          204271.0
ACTION AMBULANCE SERVICE - LYNN                                  177090.0
ACTION AMBULANCE SERVICE - STONEHAM                              431782.0
ACTION AMBULANCE SERVICE - SWAMPSCOTT                            192341.0
ACTION AMBULANCE SERVICE - WAKEFIELD                             371726.0
ACTION AMBULANCE SERVICE - WOBURN                                196171.0
ACTON FIRE DEPARTMENT STATION 1                                   67473.0
ACTON FIRE DEPARTMENT STATION 2                                   67614.0
ACTON FIRE DEPARTMENT STATION 3                                   36307.0
ACUSHNET EMERGENCY MEDICAL SERVICES                               82096.0
ACUSHNET FIRE AND RESCUE STATION 1 - HEADQUARTERS                111806.0
ACUSHNET FIRE AND RESCUE STATION 

#Shelters

In [186]:
shelt_grid = gpd.read_file('game_grid_export/facilities/shelters.shp').to_crs(grid.crs)
shelt_grid['radius'] = 3
shelt_grid['type'] = "shelter"
shelt_grid['health'] = 1
shelt_buffered = shelt_grid.copy().to_crs(grid.crs)
shelt_buffered['geometry'] = shelt_buffered['geometry'].buffer(to_m(3))
shelt_buffered['centroid'] = shelt_buffered['geometry'].centroid

shelt_info = gpd.sjoin(grid, shelt_buffered, how = 'inner', op = 'intersects')
shelt_infopop = shelt_info.groupby('NAME').sum()

alpha_shelt = shelt_buffered.sort_values('NAME').set_index('NAME')
alpha_shelt = alpha_shelt.loc[~alpha_shelt.index.duplicated(keep='first')]
alpha_shelt.loc[shelt_infopop.index,'population'] = shelt_infopop['population'].values
alpha_shelt = alpha_shelt.dropna(subset=['population'])
shelt_idx = alpha_shelt.reset_index()

alpha_shelt['population']

NAME
ABINGTON HIGH SCHOOL                                  9137.0
ABINGTON SENIOR CENTER                               14835.0
ACTON SENIOR CENTER                                   7603.0
ADAMS MIDDLE SCHOOL                                  28433.0
ADRIAN TINSLEY CENTER / BRIDGEWATER STATE COLLEGE    10563.0
AGASSIZ COMMUNITY CENTER                             58607.0
AHERN MIDDLE SCHOOL                                   7050.0
AIKEN ELEMENTARY SCHOOL                              10480.0
ALBERT F. FORD MIDDLE SCHOOL                          2726.0
ALDEN ELEMENTARY SCHOOL                               3103.0
ALFRED LIMA ELEMENTARY SCHOOL                        62965.0
AMERICAL CIVIC CENTER                                20029.0
ANGELO ELEMENTARY SCHOOL                             35089.0
ANNA M. MCCABE SCHOOL                                 7414.0
ANTHONY CARNEVALE JR. ELEMENTARY SCHOOL              48654.0
AQUINNAH TOWN HALL                                     202.0
ARLINGTON CATHOLIC 

#Fire Stations

In [187]:
fire_grid = gpd.read_file('game_grid_export/facilities/fire_stations.shp').to_crs(grid.crs)
fire_grid['radius'] = 15
fire_grid['type'] = "fire station"
fire_grid['health'] = 1
fire_buffered = fire_grid.copy().to_crs(grid.crs)
fire_buffered['geometry'] = fire_buffered['geometry'].buffer(to_m(15))
fire_buffered['centroid'] = fire_buffered['geometry'].centroid

fire_info = gpd.sjoin(grid, fire_buffered, how = 'inner', op = 'intersects')
fire_infopop = fire_info.groupby('NAME').sum()

alpha_fire = fire_buffered.sort_values('NAME').set_index('NAME')
alpha_fire = alpha_fire.loc[~alpha_fire.index.duplicated(keep='first')]
alpha_fire.loc[fire_infopop.index,'population'] = fire_infopop['population'].values
alpha_fire = alpha_fire.dropna(subset=['population'])
fire_idx = alpha_fire.reset_index()

alpha_fire['population']

NAME
ABINGTON FIRE DEPARTMENT STATION 1                             210174.0
ABINGTON FIRE DEPARTMENT STATION 2                             183927.0
ACTON FIRE DEPARTMENT STATION 1                                 67473.0
ACTON FIRE DEPARTMENT STATION 2                                 67614.0
ACTON FIRE DEPARTMENT STATION 3                                 66696.0
ACUSHNET FIRE AND RESCUE STATION 1 - HEADQUARTERS              111806.0
ACUSHNET FIRE AND RESCUE STATION 2                              79389.0
ALBION FIRE DEPARTMENT                                         183429.0
ALDEN MILLS POINT OF PINES FIRE STATION - TRAINING FACILITY    374033.0
AQUINNAH FIRE DEPARTMENT                                          826.0
ARLINGTON FIRE DEPARTMENT - HEADQUARTERS                       733870.0
ARLINGTON FIRE DEPARTMENT - HIGHLAND STATION                   667193.0
ARLINGTON FIRE DEPARTMENT - PARK CIRCLE STATION                621716.0
ASHLAND FIRE DEPARTMENT STATION 1                          

#Congregation

In [0]:
col = ['NAME', 'ID', 'geometry', 'radius', 'population', 'centroid', 'type', 'health']
hospfac = gpd.GeoDataFrame(columns = col)
EMSfac = gpd.GeoDataFrame(columns = col)
sheltfac = gpd.GeoDataFrame(columns = col)
firefac = gpd.GeoDataFrame(columns = col)


for i in col:
  hospfac[i] = hosp_idx[i]
  EMSfac[i] = EMS_idx[i]
  sheltfac[i] = shelt_idx[i]
  firefac[i] = fire_idx[i]

facdf = gpd.GeoDataFrame(columns = col)
fdf = facdf.append([hospfac, EMSfac, sheltfac, firefac])
alpha_fdf = fdf.sort_values('NAME').set_index('NAME')
fdf_idx = alpha_fdf.reset_index()

In [189]:
fdf_info = gpd.sjoin(grid, fdf, how = 'inner', op = 'intersects')
fdf_info


/usr/local/lib/python3.6/dist-packages/geopandas/tools/sjoin.py:56: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:3857', 'no_defs': True} != None)
  '(%s != %s)' % (left_df.crs, right_df.crs))


,MGRS,kmSQ_ID,GZD,EASTING,NORTHING,MGRS_10km,Shape_Leng,Shape_Le_1,Shape_Area,x_centroid,y_centroid,elevation,population_left,Health,Utility,Transporta,geometry,centroid_left,index_right,NAME,ID,radius,population_right,centroid_right,type,health
13,19TBH8319,BH,19T,283000mE,4719000mN,19TBH81,0.042368,0.042368,0.000110,-71.638932,42.597542,110.595136,48,10,7,3,POLYGON ((-7974110.544245003 5250253.962398134...,POINT (-7974809.37752691 5250912.977996822),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1
14,19TBH8419,BH,19T,284000mE,4719000mN,19TBH81,0.042368,0.042368,0.000110,-71.626755,42.597822,77.316261,91,10,7,3,POLYGON ((-7972755.143017148 5250296.197827525...,POINT (-7973453.888711865 5250955.320785713),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1
15,19TBH8519,BH,19T,285000mE,4719000mN,19TBH81,0.042368,0.042368,0.000110,-71.614578,42.598101,68.304100,28,10,7,3,POLYGON ((-7971399.722667887 5250338.238028222...,POINT (-7972098.380660096 5250997.46824583),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1
16,19TBH8619,BH,19T,286000mE,4719000mN,19TBH81,0.042369,0.042369,0.000110,-71.602401,42.598379,67.007351,39,10,7,3,"POLYGON ((-7970044.282997 5250380.082997615, -...",POINT (-7970742.853351969 5251039.420413327),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1
17,19TBH8719,BH,19T,287000mE,4719000mN,19TBH81,0.042369,0.042369,0.000110,-71.590224,42.598655,79.206305,49,10,7,3,POLYGON ((-7968688.824404924 5250421.732597118...,POINT (-7969387.306921487 5251081.177289895),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1
18,19TBH8819,BH,19T,288000mE,4719000mN,19TBH81,0.042369,0.042369,0.000110,-71.578047,42.598929,96.694630,48,10,7,3,POLYGON ((-7967333.346791559 5250463.187096115...,POINT (-7968031.741493199 5251122.738824184),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1
19,19TBH8919,BH,19T,289000mE,4719000mN,19TBH81,0.042370,0.042370,0.000110,-71.565870,42.599203,101.373822,61,10,7,3,"POLYGON ((-7965977.85025701 5250504.446220052,...",POINT (-7966676.157107949 5251164.105081731),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1
20,19TBH9019,BH,19T,290000mE,4719000mN,19TBH91,0.042370,0.042370,0.000110,-71.553692,42.599475,108.584806,48,10,7,3,POLYGON ((-7964622.334901396 5250545.509966367...,POINT (-7965320.553851569 5251205.275948916),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1
21,19TBH9119,BH,19T,291000mE,4719000mN,19TBH91,0.042370,0.042370,0.000110,-71.541514,42.599746,92.465091,80,10,7,3,POLYGON ((-7963266.800824815 5250586.378468496...,POINT (-7963964.931835451 5251246.251456492),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1
33,19TBH8118,BH,19T,281000mE,4718000mN,19TBH81,0.042364,0.042364,0.000110,-71.662901,42.587983,92.451795,70,10,7,3,POLYGON ((-7976778.730192916 5248808.699337469...,POINT (-7977477.631582461 5249467.413224254),224,HARVARD FIRE DEPARTMENT - HEADQUARTERS,10229258,15.0,44538.0,POINT (-7968917.433325215 5236550.185370948),fire station,1


#Set Cover Application For Any Single Facility


In [0]:
def calc_weight(subset, t, cost_ref):
  return sum(grid.loc[cell, 'population'] for cell in subset) / cost_ref[t]

In [0]:
 #freq_chart = [0 for _ in range(game_grid.shape[0])]
covered = set()

def set_cover(budget, facdict, cost_ref): #facdict = hosp_to_cells, EMS_to_cells, shelt_to_cells, fire_to_cells
  to_ret = set()
  while budget > 0 and len(facdict.items()) != 0:
    max_fac, max_subset = max(facdict.items(), key = lambda tup: \
                               calc_weight(tup[1], tup[0][1], cost_ref))
    
#Set cover could be wrong because of overlapping areas
#    for cell in max_subset:
#      freq_chart[cell] += 1
#       if freq_chart[cell] == 2:
#         done.add(cell)
    
    budget -= cost_ref[max_fac[1]]
    to_ret.add(max_fac)
    
    for key in facdict:
      facdict[key] -= max_subset
    del facdict[max_fac]
    
  return to_ret

In [0]:
#costs is a list of costs in the order of large hospitals, small hospitals
#large EMS, small EMS, shelters, and fire stations
def optfac (costs, budget, fac_info = hosp_info, fac = 'hospital'):
  large_hos_cost = costs[0]
  small_hos_cost = costs[1]
  large_ems_cost = costs[2]
  small_ems_cost = costs[3]
  shelter_cost = costs[4]
  fstation_cost = costs[5]

  cost_ref = {'smallhosp': small_hos_cost, 'largehosp': large_hos_cost, \
              'smallEMS': small_ems_cost, 'largeEMS': large_ems_cost, \
              'shelter': shelter_cost, 'fstation': fstation_cost}
  fac_to_cells = {}
  for i, row in fac_info.iterrows():
    fac_name = row["NAME"]
    f = ''
    if fac == 'hospital':
      f = 'smallhosp' if row['radius'] == 15 else 'largehosp'
    elif fac == 'EMS':
      f = 'smallEMS' if row['radius'] == 10 else 'largeEMS'
    elif fac == 'shelter':
      f = 'shelter'
    elif fac == 'firestation':
      f = 'fstation'
    else:
      return 'no'
    if (fac_name, f) not in fac_to_cells:
      fac_to_cells[(fac_name, f)] = {i}
    else:
      fac_to_cells[(fac_name, f)].add(i)
  
  return set_cover(budget, fac_to_cells, cost_ref)
    

In [193]:
#Example
#Costs as a list of costs in the order of large hospitals, small hospitals
#large EMS, small EMS, shelters, and fire stations; budget; facility type
#(default is hospital)
#facility options: 'hospital', 'EMS', 'shelter', 'firestation'
optfac([100, 50, 50, 25, 10, 15], 2000) 

{('ARBOUR FULLER HOSPITAL', 'smallhosp'),
 ('ARBOUR HOSPITAL, THE', 'smallhosp'),
 ('ARBOUR HUMAN RESOURCE INSTITUTE', 'smallhosp'),
 ('BAY RIDGE HOSPITAL BEVERLY', 'smallhosp'),
 ('BETH ISRAEL DEACONESS HOSPITAL - NEEDHAM', 'smallhosp'),
 ('BETH ISRAEL DEACONESS HOSPITAL MILTON', 'smallhosp'),
 ('CAMBRIDGE HEALTH ALLIANCE - SOMERVILLE CAMPUS', 'smallhosp'),
 ('CAMBRIDGE HEALTH ALLIANCE - WHIDDEN MEMORIAL CAMPUS', 'smallhosp'),
 ('CARITAS CARNEY HOSPITAL', 'smallhosp'),
 ('DANA-FARBER CANCER INSTITUTE', 'smallhosp'),
 ('EMERSON HOSPITAL', 'smallhosp'),
 ('EMMA PENDLETON BRADLEY HOSPITAL', 'smallhosp'),
 ('FAULKNER HOSPITAL', 'smallhosp'),
 ('GOOD SAMARITAN MEDICAL CENTER', 'largehosp'),
 ('HASBRO CHILDRENS HOSPITAL', 'smallhosp'),
 ('HEALTHSOUTH BRAINTREE REHABILITATION HOSPITAL', 'smallhosp'),
 ('HEALTHSOUTH NEW ENGLAND REHABILITATION HOSPITAL', 'smallhosp'),
 ('KENT HOSPITAL', 'largehosp'),
 ('LANDMARK MEDICAL CENTER', 'largehosp'),
 ('MASSACHUSETTS HOSPITAL SCHOOL', 'smallhosp'),
 (

#Set Cover Application For All Facilities

In [0]:
#costs is a list of costs in the order of large hospitals, small hospitals
#large EMS, small EMS, shelters, and fire stations
def uoptfac (costs, budget, fac_info = fdf_info):
  large_hos_cost = costs[0]
  small_hos_cost = costs[1]
  large_ems_cost = costs[2]
  small_ems_cost = costs[3]
  shelter_cost = costs[4]
  fstation_cost = costs[5]

  cost_ref = {'smallhosp': small_hos_cost, 'largehosp': large_hos_cost, \
              'smallEMS': small_ems_cost, 'largeEMS': large_ems_cost, \
              'shelter': shelter_cost, 'fstation': fstation_cost}
  fac_to_cells = {}
  for i, row in fac_info.iterrows():
    fac_name = row["NAME"]
    f = ''
    if row['type'] == "small hospital":
      f = 'smallhosp'
    elif row['type'] == "large hospital":
      f = 'largehosp'
    elif row['type'] == "small EMS":
      f = 'smallEMS'
    elif row['type'] == "large EMS":
      f = 'largeEMS'
    elif row['type'] == "shelter":
      f = 'shelter'
    elif row['type'] == "fire station":
      f = 'fstation'
    else:
      return 'no'
    if (fac_name, f) not in fac_to_cells:
      fac_to_cells[(fac_name, f)] = {i}
    else:
      fac_to_cells[(fac_name, f)].add(i)
  
  return set_cover(budget, fac_to_cells, cost_ref)
    

In [195]:
#Example
#Takes a super long time
uoptfac([100, 50, 50, 25, 10, 15], 2000, fdf_info) 

{('ABINGTON FIRE DEPARTMENT STATION 1', 'fstation'),
 ('ALDEN MILLS POINT OF PINES FIRE STATION - TRAINING FACILITY', 'fstation'),
 ('ARLINGTON FIRE DEPARTMENT - HIGHLAND STATION', 'fstation'),
 ('ARLINGTON FIRE DEPARTMENT - PARK CIRCLE STATION', 'fstation'),
 ('ATTLEBORO FIRE AND RESCUE - SOUTH ATTLEBORO STATION', 'fstation'),
 ('ATTLEBORO FIRE AND RESCUE - TWIN VILLAGE STATION', 'fstation'),
 ('ATTLEBORO FIRE DEPARTMENT - BRIGGS CORNER STATION', 'fstation'),
 ('BEDFORD FIRE DEPARTMENT', 'fstation'),
 ('BELLINGHAM FIRE DEPARTMENT STATION 3', 'fstation'),
 ('BOSTON FIRE DEPARTMENT DIVISION 1 DISTRICT 1 - ENGINE 56 LADDER 21',
  'fstation'),
 ('BOSTON FIRE DEPARTMENT DIVISION 2 DISTRICT 10 - ENGINE 30 LADDER 25',
  'fstation'),
 ('BOSTON FIRE DEPARTMENT DIVISION 2 DISTRICT 12 - ENGINE 48 LADDER 28',
  'fstation'),
 ('BOSTON FIRE DEPARTMENT DIVISION 2 DISTRICT 12 - ENGINE 52 LADDER 29',
  'fstation'),
 ('BOSTON FIRE DEPARTMENT DIVISION 2 DISTRICT 12 - ENGINE 53 LADDER 16',
  'fstation'),

#Proximity-to-Flood Function For Any Single Facility

In [0]:
flooding = gpd.read_file('game_grid_export/flooding.shp')

In [0]:
#Outputs a list of facilities in the specified distance from the flood
def prox(mindist, maxdist, flood, fac):
  fac_grid = None
  if fac == 'hospital':
    fac_grid = hospital_grid
  elif fac == 'EMS':
    fac_grid = EMS_grid
  elif fac == 'shelter':
    fac_grid = shelt_grid
  elif fac == 'firestation':
    fac_grid = fire_grid
  else:
    return "no"
  
  l = []
  for i in range(len(fac_grid)):
    dist = fac_grid.loc[i, 'geometry'].distance(flood)
    if dist <= maxdist and dist >= mindist: #TODO: mindist <= dist <= maxdist check
      l.append(fac_grid.loc[i, 'NAME'])
  return l

In [0]:
from shapely.geometry import MultiPolygon
import shapely.ops as shap
#Creates a MultiPolygon using the flood shapely file and the buffer size in meters
def flcreate(fl, b, i): 
  flooding = fl.copy().to_crs(grid.crs)
  flooding['geometry'] = flooding['geometry'].buffer(b)
  if i == 0:
    flood = flooding['geometry'].unary_union
  else:
    l = [MultiPolygon([flooding.loc[i, 'geometry'], \
                       flooding.loc[i + 1, 'geometry']]).convex_hull \
         for i in range(len(flooding) - 1)]
    flood = shap.cascaded_union(MultiPolygon(l))
    
  return flood
    

In [0]:
#Outputs all information for a list of facility names
#minimum distance, maximum distance, flood shapely file, buffer size in meters,
#type of graph (0: unary_union, 1: cascaded_union), facility type ('hospital', 'EMS', 'shelter', 'firestation')
def magic(mindist, maxdist, floodshp, buffer, graph_type = 0, fac = 'hospital'):
  l = prox(mindist, maxdist, flcreate(flooding, buffer, graph_type), fac)
  alpha_fac = None
  if fac == 'hospital':
    alpha_fac = alpha_hosp
  elif fac == 'EMS':
    alpha_fac = alpha_EMS
  elif fac == 'shelter':
    alpha_fac = alpha_shelt
  elif fac == 'firestation':
    alpha_fac = alpha_fire
  else:
    return "no"
  return alpha_fac.loc[l]

#Proximity-to-Flood For All Facilities

In [0]:
#Outputs a list of facilities in the specified distance from the flood
def uprox(mindist, maxdist, flood):
  l = []
  for i in range(len(fdf_idx)):
    dist = fdf_idx.loc[i, 'geometry'].distance(flood)
    if dist <= maxdist and dist >= mindist: #TODO: mindist <= dist <= maxdist check
      if fdf_idx.loc[i, 'NAME'] not in l:
        l.append(fdf_idx.loc[i, 'NAME'])
  return l

In [0]:
def umagic(mindist, maxdist, floodshp, buffer, graph_type = 0):
  l = uprox(mindist, maxdist, flcreate(flooding, buffer, graph_type))
  return alpha_fdf.loc[l]

In [202]:
#Example
#Takes a long time
umagic(0, 1000, flooding, 500, 0)

,ID,geometry,radius,population,centroid,type,health
NAME,,,,,,,
ABINGTON FIRE DEPARTMENT STATION 1,10218699,POLYGON ((-7888184.766090992 5180821.696749884...,10.0,100719.0,POINT (-7898184.766090994 5180821.69674988),small EMS,1
ABINGTON FIRE DEPARTMENT STATION 1,10218699,POLYGON ((-7883184.766090992 5180821.696749884...,15.0,210174.0,POINT (-7898184.766090991 5180821.696749882),fire station,1
ABINGTON FIRE DEPARTMENT STATION 2,10229091,POLYGON ((-7883150.379996872 5176937.311661702...,15.0,183927.0,POINT (-7898150.379996875 5176937.311661703),large EMS,1
ABINGTON FIRE DEPARTMENT STATION 2,10229091,"POLYGON ((-7883150.379996872 5176937.31179664,...",15.0,183927.0,POINT (-7898150.379996875 5176937.31179664),fire station,1
ACUSHNET EMERGENCY MEDICAL SERVICES,10218701,POLYGON ((-7884802.540298564 5114347.708308474...,10.0,82096.0,POINT (-7894802.540298562 5114347.70830847),small EMS,1
ACUSHNET FIRE AND RESCUE STATION 1 - HEADQUARTERS,10229092,POLYGON ((-7878448.568971436 5113658.893122129...,15.0,111806.0,POINT (-7893448.568971438 5113658.893122129),large EMS,1
ACUSHNET FIRE AND RESCUE STATION 1 - HEADQUARTERS,10229092,POLYGON ((-7878448.568971436 5113658.893122129...,15.0,111806.0,POINT (-7893448.568971438 5113658.893122129),fire station,1
ACUSHNET FIRE AND RESCUE STATION 2,10229530,POLYGON ((-7876943.304676721 5121297.445556124...,15.0,79389.0,POINT (-7891943.304676723 5121297.445556125),large EMS,1
ACUSHNET FIRE AND RESCUE STATION 2,10229530,POLYGON ((-7876943.304676721 5121297.445556124...,15.0,79389.0,POINT (-7891943.304676723 5121297.445556125),fire station,1


#Optimized Facility Flood Identification

In [0]:
def abracadabra(costs, budget, mindist, maxdist, buffer, fl = flooding, graph_type = 0, fac = 'hospital'):
  l = prox(mindist, maxdist, flcreate(fl, buffer, graph_type), fac)
  fac_info = None
  alpha_fac = None
  if fac == 'hospital':
    fac_info = hosp_info
    alpha_fac = alpha_hosp
  elif fac == 'EMS':
    fac_info = EMS_info
    alpha_fac = alpha_EMS
  elif fac == 'shelter':
    fac_info = shelt_info
    alpha_fac = alpha_shelt
  elif fac == 'firestation':
    fac_info = fire_info
    alpha_fac = alpha_fire
  else:
    return "no"

  x = optfac(costs, budget, fac_info.loc[fac_info['NAME'].isin(l)])
  l = [i[0] for i in x]
  
  return alpha_fac.loc[l]

In [0]:
def uabracadabra(costs, budget, mindist, maxdist, buffer, fl = flooding, graph_type = 0):
  l = uprox(mindist, maxdist, flcreate(fl, buffer, graph_type))

  x = uoptfac(costs, budget, fdf_info.loc[fdf_info['NAME'].isin(l)])
  l = [i[0] for i in x]
  
  return alpha_fdf.loc[l]

#Functions


##If you have no budget use magic, if you do have a budget use abracadabra, if you want to use all the facilities use umagic and uabracadabra. 

In [205]:
#Example
#magic(mindist, maxdist, floodshp, buffer, graph_type = 0, fac = 'hospital')
magic(0, 1000, flooding, 500, 0, 'hospital')

,OBJECTID,ID,ADDRESS,CITY,STATE,ZIP,TYPE,STATUS,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,WEBSITE,OWNER,TTL_STAFF,BEDS,HELIPAD,geometry,radius,type,health,centroid,population
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,
PROVIDENCE VA MEDICAL CENTER,24.0,0001102908,830 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.831937,-71.433575,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.providence.va.gov/,GOVERNMENT - FEDERAL,-999.0,73.0,NOT AVAILABLE,POLYGON ((-7936949.155586868 5135837.564535611...,15.0,small hospital,2,POINT (-7951949.155586867 5135837.564535613),338290
"NAVAL HEALTH CLINIC NEW ENGLAND, NEWPORT",27.0,0001602841,43 SMITH ROAD,NEWPORT,RI,02841,GENERAL ACUTE CARE,OPEN,NEWPORT,44005,USA,41.503931,-71.321796,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://nhcne.med.navy.mil/locations/npri/npri.asp,GOVERNMENT - FEDERAL,-999.0,100.0,NOT AVAILABLE,"POLYGON ((-7924506.027807591 5086957.87205448,...",15.0,small hospital,2,POINT (-7939506.027807593 5086957.872054482),44084
VA MEDICAL CENTER - PROVIDENCE,28.0,0013402908,830 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.832139,-71.433150,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.providence.va.gov/,GOVERNMENT - FEDERAL,-999.0,100.0,N,POLYGON ((-7936901.833110882 5135867.763963762...,15.0,small hospital,2,POINT (-7951901.833110884 5135867.763963765),337792
MEMORIAL HOSPITAL OF RHODE ISLAND,30.0,0000302860,111 BREWSTER STREET,PAWTUCKET,RI,02860,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.869922,-71.376456,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.mhri.org/,NON-PROFIT,-999.0,294.0,NOT AVAILABLE,"POLYGON ((-7920590.786097784 5141514.2622464, ...",25.0,large hospital,2,POINT (-7945590.786097785 5141514.262246399),499704
WOMEN AND INFANTS HOSPITAL OF RHODE ISLAND,32.0,0000702905,101 DUDLEY STREET,PROVIDENCE,RI,02905,WOMEN,OPEN,PROVIDENCE,44007,USA,41.810877,-71.412201,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.womenandinfants.org/,NON-PROFIT,-999.0,247.0,Y,"POLYGON ((-7924569.8250032 5132691.749874027, ...",25.0,large hospital,2,POINT (-7949569.825003202 5132691.749874027),506912
RHODE ISLAND HOSPITAL,34.0,0001002903,593 EDDY STREET,PROVIDENCE,RI,02903,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.811881,-71.409129,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.rhodeislandhospital.org/rhode-islan...,NON-PROFIT,-999.0,719.0,Y,"POLYGON ((-7924227.90673675 5132841.662859567,...",25.0,large hospital,2,POINT (-7949227.906736748 5132841.662859565),505346
EMMA PENDLETON BRADLEY HOSPITAL,35.0,0001202915,1011 VETERANS MEMORIAL PKWY,EAST PROVIDENCE,RI,02915,PSYCHIATRIC,OPEN,PROVIDENCE,44007,USA,41.792448,-71.367270,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.bradleyhospital.org/service-directo...,NON-PROFIT,-999.0,51.0,NOT AVAILABLE,POLYGON ((-7929568.200346095 5129939.701677736...,15.0,small hospital,2,POINT (-7944568.200346096 5129939.701677734),335609
HASBRO CHILDRENS HOSPITAL,37.0,0001502903,593 EDDY STREET,PROVIDENCE,RI,02903,CHILDREN,OPEN,PROVIDENCE,44007,USA,41.811150,-71.408906,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.hasbrochildrenshospital.org/,NON-PROFIT,-999.0,100.0,Y,"POLYGON ((-7934203.10357041 5132732.457129485,...",15.0,small hospital,2,POINT (-7949203.103570408 5132732.457129486),343961
ROGER WILLIAMS MEDICAL CENTER,39.0,0002002908,825 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.834839,-71.435390,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.rwmc.org/,NON-PROFIT,-999.0,177.0,NOT AVAILABLE,"POLYGON ((-7937151.19906801 5136271.097543607,...",15.0,small hospital,2,POINT (-7952151.19906801 5136271.097543609),337433


In [206]:
#Example
#umagic(mindist, maxdist, floodshp, buffer, graph_type = 0)
#Outputs all facilities
#mindist, maxdist, flood shapely, buffer, graph type
umagic(0, 1000, flooding, 500, 0)

,ID,geometry,radius,population,centroid,type,health
NAME,,,,,,,
ABINGTON FIRE DEPARTMENT STATION 1,10218699,POLYGON ((-7888184.766090992 5180821.696749884...,10.0,100719.0,POINT (-7898184.766090994 5180821.69674988),small EMS,1
ABINGTON FIRE DEPARTMENT STATION 1,10218699,POLYGON ((-7883184.766090992 5180821.696749884...,15.0,210174.0,POINT (-7898184.766090991 5180821.696749882),fire station,1
ABINGTON FIRE DEPARTMENT STATION 2,10229091,POLYGON ((-7883150.379996872 5176937.311661702...,15.0,183927.0,POINT (-7898150.379996875 5176937.311661703),large EMS,1
ABINGTON FIRE DEPARTMENT STATION 2,10229091,"POLYGON ((-7883150.379996872 5176937.31179664,...",15.0,183927.0,POINT (-7898150.379996875 5176937.31179664),fire station,1
ACUSHNET EMERGENCY MEDICAL SERVICES,10218701,POLYGON ((-7884802.540298564 5114347.708308474...,10.0,82096.0,POINT (-7894802.540298562 5114347.70830847),small EMS,1
ACUSHNET FIRE AND RESCUE STATION 1 - HEADQUARTERS,10229092,POLYGON ((-7878448.568971436 5113658.893122129...,15.0,111806.0,POINT (-7893448.568971438 5113658.893122129),large EMS,1
ACUSHNET FIRE AND RESCUE STATION 1 - HEADQUARTERS,10229092,POLYGON ((-7878448.568971436 5113658.893122129...,15.0,111806.0,POINT (-7893448.568971438 5113658.893122129),fire station,1
ACUSHNET FIRE AND RESCUE STATION 2,10229530,POLYGON ((-7876943.304676721 5121297.445556124...,15.0,79389.0,POINT (-7891943.304676723 5121297.445556125),large EMS,1
ACUSHNET FIRE AND RESCUE STATION 2,10229530,POLYGON ((-7876943.304676721 5121297.445556124...,15.0,79389.0,POINT (-7891943.304676723 5121297.445556125),fire station,1


In [207]:
#Example
#abracadabra(costs, budget, mindist, maxdist, buffer, fl = flooding, graph_type = 0, fac = 'hospital')
abracadabra([100, 50, 50, 25, 10, 15], 2000, 0, 1000, 500, flooding, 0, 'hospital')

,OBJECTID,ID,ADDRESS,CITY,STATE,ZIP,TYPE,STATUS,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,WEBSITE,OWNER,TTL_STAFF,BEDS,HELIPAD,geometry,radius,type,health,centroid,population
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,
VA MEDICAL CENTER - PROVIDENCE,28.0,0013402908,830 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.832139,-71.433150,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.providence.va.gov/,GOVERNMENT - FEDERAL,-999.0,100.0,N,POLYGON ((-7936901.833110882 5135867.763963762...,15.0,small hospital,2,POINT (-7951901.833110884 5135867.763963765),337792
SOUTHCOAST HOSPITALS GROUP - ST. LUKE'S HOSPITAL,181.0,0003602740,101 PAGE STREET,NEW BEDFORD,MA,02740,GENERAL ACUTE CARE,OPEN,BRISTOL,25005,USA,41.626400,-70.938400,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.southcoast.org/stlukes/,NON-PROFIT,-999.0,391.0,Y,"POLYGON ((-7871826.5657296 5105179.226333707, ...",25.0,large hospital,2,POINT (-7896826.565729599 5105179.226333707),152247
WOMEN AND INFANTS HOSPITAL OF RHODE ISLAND,32.0,0000702905,101 DUDLEY STREET,PROVIDENCE,RI,02905,WOMEN,OPEN,PROVIDENCE,44007,USA,41.810877,-71.412201,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.womenandinfants.org/,NON-PROFIT,-999.0,247.0,Y,"POLYGON ((-7924569.8250032 5132691.749874027, ...",25.0,large hospital,2,POINT (-7949569.825003202 5132691.749874027),506912
SOUTHCOAST HOSPITALS GROUP - CHARLTON MEMORIAL HOSPITAL,183.0,0005702720,363 HIGHLAND AVENUE,FALL RIVER,MA,02720,GENERAL ACUTE CARE,OPEN,BRISTOL,25005,USA,41.709400,-71.146000,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.southcoast.org/charlton/,NON-PROFIT,-999.0,743.0,Y,POLYGON ((-7894936.492044975 5117547.883270329...,25.0,large hospital,2,POINT (-7919936.49204497 5117547.883270328),235822
CAPE COD HOSPITAL,189.0,0005502601,27 PARK STREET,BARNSTABLE,MA,02601,GENERAL ACUTE CARE,OPEN,BARNSTABLE,25001,USA,41.653400,-70.272900,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.capecodhealth.org/hospitals-facilit...,NON-PROFIT,-999.0,259.0,NOT AVAILABLE,POLYGON ((-7797743.444570526 5109201.015920504...,25.0,large hospital,2,POINT (-7822743.444570524 5109201.015920503),75378
SOUTHCOAST HOSPITALS GROUP - TOBEY HOSPITAL CAMPUS,255.0,0005102571,43 HIGH STREET,WAREHAM,MA,02571,GENERAL ACUTE CARE,OPEN,PLYMOUTH,25023,USA,41.756100,-70.714400,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.southcoast.org/tobey/,NON-PROFIT,-999.0,100.0,Y,POLYGON ((-7856890.999769659 5124514.136134185...,15.0,small hospital,2,POINT (-7871890.999769657 5124514.136134185),29251
ROGER WILLIAMS MEDICAL CENTER,39.0,0002002908,825 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.834839,-71.435390,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.rwmc.org/,NON-PROFIT,-999.0,177.0,NOT AVAILABLE,"POLYGON ((-7937151.19906801 5136271.097543607,...",15.0,small hospital,2,POINT (-7952151.19906801 5136271.097543609),337433
DR JOHN C CORRIGAN MENTAL HEALTH CENTER,61.0,0011402720,49 HILLSIDE STREET,FALL RIVER,MA,02720,PSYCHIATRIC,OPEN,BRISTOL,25005,USA,41.707487,-71.145714,622210,"HOSPITALS, PSYCHIATRIC (EXCEPT CONVALESCENT)",NOT AVAILABLE,GOVERNMENT - STATE,-999.0,100.0,NOT AVAILABLE,POLYGON ((-7904904.706361935 5117262.615547662...,15.0,small hospital,2,POINT (-7919904.706361934 5117262.615547661),119180
RHODE ISLAND HOSPITAL,34.0,0001002903,593 EDDY STREET,PROVIDENCE,RI,02903,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.811881,-71.409129,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.rhodeislandhospital.org/rhode-islan...,NON-PROFIT,-999.0,719.0,Y,"POLYGON ((-7924227.90673675 5132841.662859567,...",25.0,large hospital,2,POINT (-7949227.906736748 5132841.662859565),505346


In [208]:
#Example
#uabracadabra(costs, budget, mindist, maxdist, buffer, fl = flooding, graph_type = 0)
#Outputs all facilities + set cover
#costs, budget, mindist, maxdist, buffer, flood shapely, graph type
uabracadabra([100, 50, 50, 25, 10, 15], 2000, 0, 1000, 500, flooding, 0)

,ID,geometry,radius,population,centroid,type,health
NAME,,,,,,,
BOSTON FIRE DEPARTMENT DIVISION 2 DISTRICT 8 - ENGINE 16,10229140,POLYGON ((-7902450.033897113 5202754.848867148...,10.0,366531.0,POINT (-7912450.03389711 5202754.848867146),small EMS,1
BOSTON FIRE DEPARTMENT DIVISION 2 DISTRICT 8 - ENGINE 16,10229140,POLYGON ((-7897450.033897113 5202754.848731843...,15.0,614489.0,POINT (-7912450.033897116 5202754.848731842),fire station,1
CHELSEA FIRE DEPARTMENT - ENGINE 3/LADDER 2,10229410,POLYGON ((-7896192.590091074 5220969.776106819...,10.0,348841.0,POINT (-7906192.590091074 5220969.776106817),small EMS,1
CHELSEA FIRE DEPARTMENT - ENGINE 3/LADDER 2,10229410,POLYGON ((-7891192.590091074 5220969.776106819...,15.0,682214.0,POINT (-7906192.590091076 5220969.776106821),fire station,1
DARTMOUTH FIRE DISTRICT 3 STATION 3,10491467,POLYGON ((-7886781.886951506 5107175.801447784...,15.0,113769.0,POINT (-7901781.886951505 5107175.801447785),fire station,1
DARTMOUTH FIRE DISTRICT 3 STATION 3,10491467,POLYGON ((-7886781.886951506 5107175.801447784...,15.0,113769.0,POINT (-7901781.886951505 5107175.801447785),large EMS,1
PEMBROKE FIRE DEPARTMENT STATION 3,10229372,POLYGON ((-7865397.178962265 5174200.259962173...,15.0,79045.0,POINT (-7880397.178962265 5174200.259962173),fire station,1
HOLBROOK FIRE DEPARTMENT - HEADQUARTERS,10229320,POLYGON ((-7889694.554788984 5182899.395221852...,15.0,235792.0,POINT (-7904694.554788985 5182899.395221853),fire station,1
HOLBROOK FIRE DEPARTMENT - HEADQUARTERS,10229320,POLYGON ((-7894694.554788984 5182899.395221852...,10.0,126255.0,POINT (-7904694.554788982 5182899.39522185),small EMS,1
